In [1]:
# Necessary for importing modules from a sub-directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if os.getcwd().split('/')[-1] == 'notebooks':
    print("CHANGE DIR TO ROOT")
    os.chdir(r"../")

import pickle
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import csv
import time
from datetime import datetime
import os

from preprocess import article_text_to_tfidf, build_ne_gold_phraser, get_gold_ngrams, apply_ngrams
from evaluation import aggregate_metrics, get_metric_umass, get_metric_cv
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora

from efselabwrapper import run_annotation_pipeline, run_processing_pipeline, run_processing_pipeline_ner

CHANGE DIR TO ROOT
Already downloaded a model for the 'sv' language


In [2]:
if 1==1:
    DATASET_TYPE = 'DN'
    DATASET_SIZE = '10000'
    DATASET_INDEX = '1'
    data_dir = f'var/data_prod_noun_propn'
    corpus = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_{DATASET_INDEX}.pkl', 'rb'))

else:
    raw_df = pickle.load(open(f'var/data/DN_10000_1.pkl', 'rb'))
    raw_strings = [row[1].content_text for row in raw_df.iterrows()]

    pos_tags = ['NOUN']
    corpus = run_processing_pipeline(raw_strings, pos_tags)
    
ext_corpus = pickle.load(open(f'{data_dir}/efselab_extrinsic_20000.pkl', 'rb'))
NUM_ARTICLES = len(corpus)
print(f'NUMBER OF ARTICLES IN CORPUS: {NUM_ARTICLES}')

NUMBER OF ARTICLES IN CORPUS: 10000


In [3]:
c1 = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_1.pkl', 'rb'))
c2 = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_2.pkl', 'rb'))
c3 = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_3.pkl', 'rb'))
c4 = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_4.pkl', 'rb'))
c5 = pickle.load(open(f'{data_dir}/efselab_{DATASET_TYPE}_{DATASET_SIZE}_5.pkl', 'rb'))
corpus = c1 + c2 + c3 + c4 + c5
print(len(corpus))

50000


In [4]:
flat = [e for sublist in corpus for e in sublist]
x = [e for e in flat if 'åring' == e]
len(x)

0

In [5]:
#tpe = 'DN' # DN, BN, DN+BN+HD
#corpus = pickle.load(open(f'var/data_survey/efselab_parsed_{tpe}_10000_survey.pkl', 'rb'))
#print(len(corpus))

#a1 = corpus[10000]; a2 = corpus[10001]; a3 = corpus[10002] 
#tp = [e for e in corpus[:10000] if e == a1 or e == a3 or e == a2]
#tp

In [6]:
corpus = pickle.load(open(f'var/data/preprocessed_NOUN_prod_4_DEC.pkl', 'rb'))

In [7]:
CLASSIFICATION = False
if CLASSIFICATION:
    test_data = pickle.load(open(f'var/data/efselab_parsed_{DATASET_TYPE}_4000_1.pkl', 'rb'))
    test_data2 = pickle.load(open(f'var/data/efselab_parsed_{DATASET_TYPE}_4000_2.pkl', 'rb'))
    test_data3 = pickle.load(open(f'var/data/efselab_parsed_{DATASET_TYPE}_4000_3.pkl', 'rb'))
    test_data4 = pickle.load(open(f'var/data/efselab_parsed_{DATASET_TYPE}_4000_4.pkl', 'rb'))
    test_data5 = pickle.load(open(f'var/data/efselab_parsed_{DATASET_TYPE}_4000_5.pkl', 'rb'))
    test_truncated = test_data + test_data2 + test_data3 + test_data4 + test_data5

In [45]:
# Preprocess varibles
MAX_DF = 0.95 # float (0.95)
MIN_DF = 2    # int: 2
NGRAM = False # False | True
if NGRAM is True:
    NGRAM_OPT = 'gold+ne' # 'ext+int' | 'int' 
else:
    NGRAM_OPT = '-'

# Model variables

#NUM_ARTICLES = len(df.index)
NUM_ARTICLES = len(corpus)
NUM_TOPICS = 40

NMF_NORM = 'frobenius' # ‘frobenius’ | ‘kullback-leibler’ | ‘itakura-saito’
NMF_SOLVER = 'mu' # ‘cd’ | ‘mu’
MAX_ITERATIONS = 500
INIT = None # None | ‘random’ | ‘nndsvd’ | ‘nndsvda’ | ‘nndsvdar’ | ‘custom’
ALPHA = 0 # float
L1_RATIO = 0.0 # float

MODEL_NAME = f'SKL_NMF_{NMF_NORM}_{NMF_SOLVER}'
if INIT is not None:
    MODEL_NAME = f'{MODEL_NAME}_init-{str(INIT)}'
if (ALPHA != 0):
    MODEL_NAME = f'{MODEL_NAME}_alpha-{ALPHA}_L1-{L1_RATIO}'
print(MODEL_NAME)


SKL_NMF_frobenius_mu


In [46]:
if (NGRAM is True):
    # Load NE_grams and gold_grams
    NE_GRAMS_FILE = f'var/ne_ngrams_{DATASET_TYPE}_{DATASET_SIZE}_{DATASET_INDEX}.pkl'
    ne_ngrams = pickle.load(open(NE_GRAMS_FILE, 'rb'))
    gold_ngrams = get_gold_ngrams()
    empty_grams = {
        'bigrams': [],
        'trigrams': []
    }

    if NGRAM_OPT == 'gold+ne':
        phraser = build_ne_gold_phraser(ne_ngrams, gold_ngrams)
    else:
        phraser = build_ne_gold_phraser(empty_grams, gold_ngrams)
        
    corpus = apply_ngrams(corpus, phraser)
    ext_corpus = apply_ngrams(ext_corpus, phraser)


In [47]:
# Create TF-IDF
vectorizer = TfidfVectorizer(
    lowercase=False,
    tokenizer=lambda x: x,
    max_df=MAX_DF, 
    min_df=MIN_DF, max_features=None, use_idf=True)
tf_idf = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()

NUM_FEATURES = len(feature_names)
print(f'#Features from tfidf is {NUM_FEATURES}')

#Features from tfidf is 128401


In [48]:
# Fit model
t_start = time.time()

model = NMF(random_state=0, tol=0.0001, verbose=0, shuffle=False,\
            n_components=NUM_TOPICS, beta_loss=NMF_NORM, solver=NMF_SOLVER, \
            init=INIT, alpha=ALPHA, l1_ratio=L1_RATIO, max_iter=MAX_ITERATIONS)

W = model.fit_transform(tf_idf, y=feature_names)
H = model.components_
ITERATIONS = model.n_iter_

RUN_TIME = round(time.time() - t_start, 3)

In [49]:
if CLASSIFICATION:
    # Apply model as classifier for unseen data
    test_tf_idf = vectorizer.transform(test_truncated)
    W = model.transform(test_tf_idf)

In [50]:
# Get list of topics (nested list)
def get_NMF_topics(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return topics

N_TOP_WORDS = 10
topics = get_NMF_topics(model, feature_names, N_TOP_WORDS)
#topics_full = get_NMF_topics(model, feature_names, 10)

In [51]:
#tvar = [[e for e in t if e not in [w for sl in topics_full[:i]+topics_full[i+1:] for w in sl]] for i,t in enumerate(topics_full)]
#print(tvar)
#topics_tf_idf = vectorizer.transform(tvar+[])
#W_test = model.transform(topics_tf_idf) 

#article_topics = {}
#for i in range(len(W_test)):
#    tt = sorted([(f'T{k}', f'{num:.5f}') for k,num in enumerate(W_test[i])], key=lambda x: x[1], reverse=True)[:5]
#    article_topics[i] = tt
#    print(f'Doc{i}: ', tt)

In [52]:
#topics = []
#for topic_idx, topic in enumerate(model.components_):
#    topics.append([feature_names[i] for i in np.flip(topic.argsort()[-11:])])
#df_topic_log = pd.DataFrame(topics)
#tsave = f'metric_log/topics/df_topics_{DATASET_TYPE}{DATASET_SIZE}_{DATASET_INDEX}_{WC}'
#df_topic_log.to_csv(tsave+'.csv')

In [53]:
# Format Gensim components for Coherence metrics
DO_EVAL = False
if DO_EVAL:
    ext_gensim_dict = corpora.Dictionary(ext_corpus)

    tok2id = ext_gensim_dict.token2id
    flat_topic_terms = [term for topic in topics for term in topic]
    topic_terms_not_in_ext_corpus = [[term] for term in flat_topic_terms if term not in tok2id]

    smooth_corpus = ext_corpus + topic_terms_not_in_ext_corpus
    smooth_gensim_dict = corpora.Dictionary(smooth_corpus)
    print(f'{len(topic_terms_not_in_ext_corpus)} number of terms not in extrinsic corpus added as separate one-term documents')

    int_gensim_dict = corpora.Dictionary(corpus)
    int_gensim_corpus = [int_gensim_dict.doc2bow(text) for text in corpus]

In [54]:
# Calculate Coherence metrics
if DO_EVAL:
    m_cv = get_metric_cv(topics, smooth_corpus, smooth_gensim_dict)
    coh_cv = [e[0] for e in m_cv]
    std_cv = [e[1] for e in m_cv]
    m_umass = get_metric_umass(topics, int_gensim_dict, int_gensim_corpus)
    coh_umass = [e[0] for e in m_umass]
    std_umass = [e[1] for e in m_umass]

    reduced_cv = aggregate_metrics(coh_cv)
    reduced_cv['std'] = round(np.mean(std_cv), 4)
    reduced_umass = aggregate_metrics(coh_umass)
    reduced_umass['std'] = round(np.mean(std_umass), 4)

In [55]:
# Calculate article assignment distinction
#article_assignment_metric = [sum(sorted([num for num in W[i]], reverse=True)[:3]) for i in range(len(corpus))]
#reduced_a = aggregate_metrics(article_assignment_metric)
#print(reduced_a)

In [56]:
# Calculate document-topic relative sparseness
def doc_top_relative_sparseness(W, top_n):
    res = []
    for i in range(len(W[:])):
        #topics = sorted([num for num in W[i]], reverse=True)    
        topics = sorted(W[i], reverse=True)    
        res.append(sum(topics[:top_n])/sum(topics))
    return res

top_n_topics = 1

#relative_sparseness = doc_top_relative_sparseness(W, top_n_topics)
relative_sparseness = [sum(sorted(W[i], reverse=True)[:top_n_topics])/(sum(W[i]) if sum(W[i]) > 0 else 1) for i in range(len(W[:]))]
reduced_rs = aggregate_metrics(relative_sparseness)
print(reduced_rs)

{'avg': 0.5007, 'med': 0.4694, 'top': 0.872, 'bot': 0.2429}


In [57]:
print('MODEL STATISTICS\n')
print(MODEL_NAME)
print('# Articles: ', NUM_ARTICLES)
print('# Topics:', NUM_TOPICS)
print('NGRAM = True') if NGRAM == 'True' else print('NGRAM = False')
print('Iterations:', ITERATIONS)
print('Runtime: ', RUN_TIME, 'seconds')
print()

if DO_EVAL:
    print('C_v:\t ', reduced_cv)
    print('U_mass:\t ', reduced_umass)
    print(f'RS ({top_n_topics}):\t ', reduced_rs)

MODEL STATISTICS

SKL_NMF_frobenius_mu
# Articles:  110034
# Topics: 40
NGRAM = False
Iterations: 30
Runtime:  72.079 seconds



In [58]:
# Print topics with N top words
print_topics = True

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        #message = "Topic #%d: " % topic_idx
        message = str(topic_idx+1) +': '
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        #message = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message, '\n')

if print_topics:
    print_top_words(model, feature_names, 10)
    

1: fråga jobb arbete verksamhet peng land samhälle kris möjlighet problem 

2: hus kvadratmeter pris krona ägare boyta säljare ägarbyte adress fastighet 

3: match poäng lag seger halvlek mål tränare säsong förlust spel 

4: polis presstalesperson plats händelse brott natt larm gärningsman vittne skada 

5: miljon krona kvartal resultat rörelseresultat omsättning vinst skatt nettoomsättning aktie 

6: patient vård läkare sjukvård sjuksköterska personal region vårdcentral hälsa behandling 

7: olycka räddningstjänst väg bil ambulans trafik personbil larm trafikolycka plats 

8: president land protest val premiärminister demonstrant demokrat ledare uttalande presidentval 

9: barn förälder familj förskola mamma skola pappa flicka liv pojke 

10: kommun boende kommunstyrelse äldreboende verksamhet ordförande äldreomsorg hemtjänst förskola område 

11: säsong klubb spelare kontrakt tränare sportchef liga division trupp spel 

12: aktie bolag stockholmsbörs börs kurs riktkurs finansinspekti

In [44]:
print(topics[73])

IndexError: list index out of range

In [ ]:
thresholded_articles = []
threshold = np.max(W)*0.1
for i in range(len(W)):
    topics_for_doc = sorted([(f'T{k}', num) for k,num in enumerate(W[i])], key=lambda x: x[1], reverse=True)[:5]
    if topics_for_doc[0][1] > threshold:
        entry = (f'Doc{i}', [(e[0], f'{e[1]:.5f}') for e in topics_for_doc])
        thresholded_articles.append(entry)
print(len(thresholded_articles))


In [ ]:
thresholded_articles[:100]

In [ ]:
article_topics = {}
max_w, w_i = 0, 0
for i in range(len(W)):
    topics_for_doc = sorted([(f'T{k}', f'{num:.5f}') for k,num in enumerate(W[i])], key=lambda x: x[1], reverse=True)[:5]
    article_topics[i] = topics_for_doc
    if float(topics_for_doc[0][1]) > max_w:
        max_w = float(topics_for_doc[0][1])
        w_i = i
    if i <= 100: 
        print(f'Doc{i}: ', topics_for_doc)
print(max_w, w_i)

In [ ]:
#article_topics = {}
for i in range(len(W)):
    topics_for_doc = sorted([(k, num) for k,num in enumerate(W[i])], key=lambda x: x[1], reverse=True)
    if i >= 10000:
        s = sum([e[1] for e in topics_for_doc])
        m1 = f'({" ".join(topics[topics_for_doc[0][0]])})*{round(topics_for_doc[0][1]/s, 3)}'
        m2 = f'({" ".join(topics[topics_for_doc[1][0]])})*{round(topics_for_doc[1][1]/s, 3)}'
        m3 = f'({" ".join(topics[topics_for_doc[2][0]])})*{round(topics_for_doc[2][1]/s, 3)}'
        print(m1 + ', ' + m2 + ', ' + m3)
    

In [ ]:
corpus[19]

In [ ]:
# Save model to csv
if False:
    METRIC_FILENAME = f'metric_log/nmf_metrics_data={DATASET_TYPE}{NUM_ARTICLES}_single_2.csv'
    DATE = datetime.now().strftime('%m%d-%H:%M')

    fields=[DATE, MODEL_NAME, NUM_ARTICLES, NUM_TOPICS, ITERATIONS, RUN_TIME, NGRAM_OPT, NUM_FEATURES, \
            reduced_cv.get('avg'), reduced_cv.get('top'), reduced_cv.get('bot'), \
            reduced_umass.get('avg'), reduced_umass.get('top'), reduced_umass.get('bot'), \
            reduced_rs.get('avg'), reduced_rs.get('top'), reduced_rs.get('bot'), '_']

    if os.path.exists(METRIC_FILENAME):  
        with open(METRIC_FILENAME, 'a+', newline='\n', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerow(fields)
        f.close()
    else:
        with open(METRIC_FILENAME, 'w', newline='\n', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',')
            headers=['date', 'model', '#articles', '#topics', 'iterations', 'time', 'ngram', '#features', \
            'cv_avg', 'cv_top', 'cv_bot', \
            'umass_avg', 'umass_top', 'umass_bot', \
            'rs_avg', 'rs_top', 'rs_bot', \
             '_']
            writer.writerow(headers)
            writer.writerow(fields)
        f.close()
